In [2]:
import traceback
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import os

In [3]:
# Extraction function
def tflog2pandas(path):
    runlog_data = pd.DataFrame({"metric": [], "value": [], "step": []})
    try:
        event_acc = EventAccumulator(path, size_guidance={"scalars": 0})
        event_acc.Reload()
        tags = event_acc.Tags()["scalars"]
        for tag in tags:
            event_list = event_acc.Scalars(tag)
            values = list(map(lambda x: x.value, event_list))
            step = list(map(lambda x: x.step, event_list))
            r = {"metric": [tag] * len(step), "value": values, "step": step}
            r = pd.DataFrame(r)
            runlog_data = pd.concat([runlog_data, r])
    # Dirty catch of DataLossError
    except Exception:
        print("Event file possibly corrupt: {}".format(path))
        traceback.print_exc()
    return runlog_data


In [57]:
# path="/home/test/sampath/flda/output/runs/train_2020_real_to_clipart_infograph_quickdraw_painting_sketch_VITs_9900133" #folderpath
# path="/home/test/sampath/flda/output/runs/train_2020_real_to_clipart_infograph_quickdraw_painting_sketch_VITs_9911133" #folderpath
# path = '/home/test/sampath/flda/output/runs/train_2020_sketch_to_clipart_infograph_quickdraw_painting_real_VITs_9900133'
path = '/home/test/sampath/flda/output/runs/train_2020_sketch_to_clipart_infograph_quickdraw_painting_real_VITs_9911133'
df=tflog2pandas(path)
df.set_index('metric', inplace=True)

In [58]:
import numpy as np

row = df.loc["test_average_of_5_domains/Accuracy"]
arr = row['value'].to_numpy()

print(len(arr))
best_index = np.argmax(arr)  # Index in NumPy array
print("Best index in array:", best_index)
avg = arr[best_index]
print(avg)

50
Best index in array: 16
83.26338958740234


Best step: value    test_average_of_5_domains/Accuracy
step     test_average_of_5_domains/Accuracy
dtype: object
Best accuracy: value    76.250458
step     22.000000
dtype: float64


In [59]:
# doms = ['infograph','clipart','painting','sketch','quickdraw']
doms = ['infograph','clipart','painting','real','quickdraw']
results = {}
avg_accu = 0
for dom in doms:
    # col = f'test_unsup_{dom}/Accuracy'
    col = f'clientside_test_average_of_{dom}:domain/Accuracy'
    rows_d = df.loc[col]
    arr = rows_d['value'].to_numpy()
    results[dom] = arr[best_index]
    avg_accu += arr[best_index]
print('avegrage accuracy is ', avg_accu/5)


avegrage accuracy is  83.26339263916016


In [60]:
results

{'infograph': 60.20915985107422,
 'clipart': 88.77900695800781,
 'painting': 94.78054809570312,
 'real': 96.50824737548828,
 'quickdraw': 76.04000091552734}